

![figure](../lab2/lab2_figures/politecnico_h-01.png)



# **Eletrónica Configurável / Configurable Electronics**
#### Mestrado em Engenharia Eletrotécnica / Master in Electrical and Electronic Engineering

## **LabWork2 - Vivado Synthesis and Implementation Tools**

_______

## Introduction ##
This lab provides an in-depth tour to simulation, synthesis and implementation processes. You will analyze the design and the generated reports; perform static timing analysis; implement the design with the default settings; and generate a bitstream. Then you will open a hardware session and program the FPGA.

### Objectives ###
After completing this lab, you will be able to:

* Create a Vivado project sourcing IPs from the IP Catalog and targeting a specific FPGA device
* Run the simulator in Vivado IDE
* Display signal waveforms with analog viewer and debug the design
* Synthesize the design with the provided basic timing constraints and analyze the output 
* Change the synthesis settings and see their effects on the generated output
* Implement the design, generate various reports and analyze the results
* Generate bitstream and verify the functionality in hardware


In the instructions below **{sources}** refers to `C:\Xilinx\MEE_EC\sources` and **{labs}** refers to `(C:\Xilinx\MEE_EC\labs)`

### Design description ###

The block diagram shown in the following figure shows the structure of one possible system that can be used to generate a PWM signal, modulated using a sine wave with two different frequencies. The frequency chosen depends on the position of the two-state on-board switch.

![Figure](../lab2/lab2_figures/fig1.png)


_____

## Step 1 - Create a Vivado Design ##

### Step 1.1 ###

In this section you will create a new and empty project for PYNQ-Z2 Board in Vivado. We will use Vivado 2020.1 - note that some options may vary depending on the version you are using.

* Start downloading the source files provided (Moodle) and save them in `{sources}/lab2`.


* Open Vivado and create a new project to start the wizard. Click **Next**.


* Click the browse button of the project location field of the new project form, browse to `c:/Xilinx/MEE_EC/labs` and click **Select**. Enter `lab2` in the project name field. Make sure that the *Create Project Subdirectory* box is checked. Click **Next**.


* We will create an empty RTL project and specify sources later. Select **RTL Project** option in the *Project Type* form, and click **Next**. Make sure that the *Do not specify sources* at this time box is checked.


* In the *Default Part* form, select the *Boards* option and choose the **PYNQ-Z2** board. Click **Next** and then **Finish** to create the Vivado project.


### Step 1.2 ### 

In this section you will add some sources to the project.

* In the *Sources* pane select **Add Sources** by clicking the **plus** sign. Then select *Add or create design sources* and click **Next**.


* Select *Add Files* and browse to the `{sources}\lab2` directory, and select simultaneously the following data and verilog files: `counter.v`, `ftrigger.v`, `pwm.v`, `sine.v` and `sine_values.dat`. Click **OK** and make sure these files are added to the project (check optional box). Click **Finish**.


* Repeat the last two steps but now select *Add or create constraints* and click **Next**. Select *Add Files* and browse to the `{sources}\lab2` directory, select `lab2.xdc`, click **OK** and then **Finish**. 

### Step 1.3 ### 

We will now create a top level block design named lab2. 

* In the *Flow Navigator*, expand *IP Integrator* and select **Create Block Design** command. In the dialog box, specify **lab2** name in the *Design name* field and click **OK**.


* The Vivado IDE will display a blank *Diagram* canvas. In the *Sources* pane select the `pwm.v` file and **drag it** to the diagram. Do the same for the other three verilog modules: `ftrigger.v`, `sine.v` and `counter.v`.


* We need now to configure each of the IPs in the block design. Double-click on the each of the IP cores to re-customize them according to the following: **counter** Value=255 and Depth=8; **sine** Depth=8 and Width=12; **pwm** Width=12. After we re-customize all three IPs, the IP Integrator canvas should look like:

![Figure](../lab2/lab2_figures/fig3.png)


* The last IP necessary for our design is the *Constant* IP core. Add it four times into the block design. Two *Constant* IP instances will be connected to the **div_factor_freqhigh(31:0)** and **div_factor_freqlow(31:0)** ports of the `ftrigger_0` module and the remaining two instances to the **div_factor_freqhigh(31:0)** and **div_factor_freqlow(31:0)** ports of the `pwm_0` module.


* Double-click on the *xlconstant_0* module and set **Const Width = 32** and **Const Value = 110592**. 

<div class="alert alert-block alert-info">
<b>Info:</b> Const Width = 32 because *div_factor_freqhigh* port that we would like to connect to is 32-bit wide. Const Value = 110592 because this is the number that divides the frequency of the input clock signal (100 MHz) to the required frequency.
</div>

* Do the same procedure with the *xlconstant_1* module. Set **Const Width = 32** and **Const Value = 389120**.


* Do the same procedure with the *xlconstant_2* module. Set the **Const Width = 32** and **Const Value = 27**.

<div class="alert alert-block alert-info">
<b>Info:</b> Const Value = 27 (110592/4096=27), because PWM module must operate at 2^width (2^12=4096) higher frequency then the Sine module. This is required in order to generate correct pwm signal.
</div>

* In the forth module *xlconstant_3*, set **Const Width = 32** and **Const Value = 95 (389120/4096=95)**.


* Finally, add a *Clocking Wizard* IP core, double click it and select the *Clocking Options* tab. Set the input clock fequency to **125 MHz**. Leave all the other configurations as default. In the *Output Clocks* tab make sure that the clk_out1 clock is set to **100 MHz**. Then click **OK** to costumize the module. 

### Step 1.4 ### 

We will now connect the IPs between themselves. 

* The *Designer Assistance* is available so you can click the **Run Connection Automation**. Select all and click **OK**. Vivado will make the clock and reset input connections. Change the reset port name to **reset**.


* Select **sw0** pin of *pwm_0* module, right-click on it and select *Create port...* option. In the *Create Port* dialog box, make sure the *Connect to ‘sw0’ seleted pin* is checked, change the name to **sw** and leave all other parameters unchanged. Click **OK**.


* Repeat the same procedure for **pwm_out** pin.


* Now you need to connect the IPs. To do that, place the cursor on top of the **clk_out1** pin of *clk_wiz_0* module and you can notice that the cursor changes into a pencil indicating that a connection can be made from that pin. Clicking the left mouse button a connection starts. Click and drag the cursor to the **clk_in** pin of *ftrigger_0* module. 

<div class="alert alert-block alert-info">
<b>Info:</b> You must press and hold down the left mouse button while dragging the connection from one pin to another. As you drag the connection wire, a green checkmark appears on the port indicating that a valid connection can be made between these points. In fact, Vivado highlights all possible connections points in the subsystem design as you interactively wire the pins and ports. Release the left mouse button and Vivado makes connection between desired ports. 
</div>

* Repeat this procedure until all the pins are connected as shown in the figure. Click **Validate Design** and then **Regenerate Layout**. You should have a design like the one shown in the figure.


![Figure](../lab2/lab2_figures/fig5.png)


* Use the `File -> Save Block Design` command from the main menu to **save** the design.


* In the *Sources* window, select `lab2`, right-click on it and choose **Create HDL Wrapper...** option, letting Vivado manage the wrapper ans auto-update.


* Under *IP Integrator*, click *Generate Block Design* and select **Generate**. The option *out of context per IP* can significantly reduce synthesis run times because the IP cache can be used with this option to prevent Vivado synthesis from regenerating output products for specific IP if they do not change.


### Step 1.5 ###

We will now synthetize, implement and test the design.

* Vivado synthesises and implements the module defined as top module. To make sure your design is the top module, right-click on `lab2_wrapper` on the *Sources* window and select **Set as Top** from the dropdown menu. The wrapper file should now appear in the *Sources* window with a top level symbol.

![Figure](../lab2/lab2_figures/fig6.png)


* Synthesize your design with **Run Synthesis** option from the *Flow Navigator*. Your synthesized design should now include input and output buffers.


* Implement your design with **Run Implementation** option from the *Flow Navigator*.


* Generate bitstream file with **Generate Bitstream** option from the *Flow Navigator*. Program your Pynq-Z2 device and observe the pwm signal in LED0, with different frequencies, depending on SW0.


_____

## Step 2 - Explore Synthesis and Implementation## 

### Step 2.1 ###

We will Synthesize the design and analyze the Reports.

* Start by adding a new timing constraints file. Browse to the `{sources}\lab2` directory, select `lab2_timing.xdc`, click **OK** and then **Finish**. 


* In the *Sources* pane, expand the *Constraints folder* and double-click the `lab2_timing.xdc` entry to open the file in text mode.


<div class="alert alert-block alert-info">
<b>Info:</b> Line 5 creates the period constraint of 8ns with a duty cycle of 50%. Line 13 creates a virtual clock with period 12ns. This clock can be viewed as the upstream device board clock (the upstream device generates its output with respect to the board clock). The reset pin is constrained with respect to the system clock (lines 23, and 24) whereas the sw pin is constrained with respect to the upstream clock (lines 28, 29). The pwm_out pin is constrained with respect to the upstream clock as the downstream device may be using it. 
</div>


* Click on **Run Synthesis** under the *Synthesis tasks* of the *Flow Navigator* pane. When the process is completed sSelect the *Open Synthesized Design* option and click **OK** as we want to look at the synthesis output.


* Click on **Report Noise** under the *Open Synthesized Design* entry of the *Synthesis* tasks of the *Flow Navigator* pane. Click **OK** to generate the report named **ssn_1**. The noise report opens at the lower window. Note that the noise analysis is done only on the output pins and the margins are very good (99,83%) as SSN induced problems are not expected in a design with a single output.


* Click on **Report Timing Summary** under the *Open Synthesized Design* entry of the *Synthesis* tasks of the *Flow Navigator* pane. Leave all the settings unchanged, and click **OK** to generate a default timing report named **timing_1**.

![Figure](../lab2/lab2_figures/fig7.png)

<div class="alert alert-block alert-info">
<b>Info:</b> Notice that the Design Timing Summary and Inter-Clock Paths entry in the left pane is highlighted in red indicating timing violations. In the right pane, the information is grouped in Setup, Hold, and Width columns. Under the Setup column Worst Negative Slack (WNS) is linked indicating that clicking on it can give us insight on how the failing path has formed. The Total Negative Slack (TNS) is highlighted in red indicating the total amount of violations in the design and the Number of Failing Endpoints indicate total number of failing paths. 
</div>


* Click on the **Worst Negative Slack (WNS)** link and note that you have: 
    * 10 input failing paths (Path 23 to 31), from *sw* port to *ftrigger_0* module; 
    * 1 output failing path (Path 21), from *pwm_0* module to *pwm_out* port.


* Select **Path 21** in the timing summary panel, right-click, and select **Schematic**. The schematic for the data path will be displayed.


* In order to see how the *Source Clock Path* is made up in schematic form, double-click on left end of the **C** pin of the FDRE in the schematic. This will show the net between the BUFG and C port of the FDRE. Similarly, double-click on the left end of the BUFG to see the path between MMCM and BUFG. Double click **CLKIN1** pin of the MMCM and continue until you find the **sys_clock** port. This corresponds to the *Source Clock Path* in the timing report.


![Figure](../lab2/lab2_figures/fig8.png)


* Double-click on the **Path 21** to see how the path is made. The path report shows three sections: 
    * Summary
    * Source Clock Path
    * Data Path
    * Destination Clock Path.


<div class="alert alert-block alert-info">
<b>Info:</b> The rise time instant for the input clock is 10ns. Considering the delay introduced by the input buffer, MMCM, routing nets and global buffer is (1.451ns + (-5,113ns) + 0.8ns + 0.101ns + 0.584ns), the rising edge arrives the flip-flop (FDRE) at 8.407ns. The flip-flop adds 0.478ns clock to Q delay, followed by 0.8ns net delay and 3.705ns OBUF delay, so the arrival time is 13.389ns. 
We are assuming that this pwm_out pin will be clocked externally by a board clock with 12ns period. Considering that this clock period may have an uncertainty of -0.164ns, data should be available at time 11.836ns, so that it can be captured by the rising edge of this virtual clock. However, it will arrive only at 13.389ns, which corresponds to a slack of -1.553ns.
</div>

<div class="alert alert-block alert-warning">
<b>Attention:</b> these are estimates only! The nets are specified as unplaced and have all been allocated default values (0.800 ns). No actual routing delays are considered.
</div>


* Click **Edit Timing Constraints** under the *Synthesized Design*. In the *Timing Constraints* GUI 3-click in the *Period* cell of the **virtual_clock** and change the period from 12 to 10. Click **Apply**. 


* Since the timing constraint has changed, a warning message in the console pane is displayed to rerun the report.Click on **Rerun**. Notice that setup timing violations are gone. 


* Select `File > Save Constraints`.	Click **OK** and then click **Yes** to save the synthesized design. Notice that the **.xdc** file has been updated and Synthesis **Out-of-Date** status is displayed on the top-right corner. Click on the link to update the synthesis process.


* Explore the **Report Utilization** and the **Report Power** under the *Open Synthesized Design* entry of the *Synthesis* tasks of the *Flow Navigator* pane. Note however that these are just estimates. After implementation all these reports will contain actual data regarding the design implementation on this FPGA device.


### Step 2.2 ###

Now, we will Implement the design and analyze the Reports.

* Click on the **Run Implementation** in the *Flow Navigator* pane. Click **OK**. When the implementation is completed, a dialog box will appear with three options. Select the *Open Implemented Design* option and click **OK**.


* In the *Flow Navigator* pane, select `Open Implemented Design > Report Utilization`. The Report Utilization dialog box opens. Click **OK**. The utilization report is displayed at the bottom of the Vivado IDE. You can select any of the resources on the left to view its corresponding utilization.


* In the *Flow Navigator*, select `Implementation Design > Report Timing Summary `. The Report Timing Summary dialog box opens. Leave all the settings unchanged and click **OK** to generate the report. 


* Click on the **Worst Negative Slack (WNS)** link and double-click **Path 21** again. Inspect the delays in the *Source Clock Path*, *Data Path* and *Destination Clock Path*. Note that net delays are no longer estimates and reflect the design placement and routing.

<div class="alert alert-block alert-info">
<b>Info:</b> Note that the arrival time is 5.232ns and the clock period is 10ns. This means that the design will work just fine, with a positive slack of 4.604ns (considering a clock uncertainty of 0.164ns). However, if you increased the clock frequncy to 200MHz (T=5ns) this path would probably fail timing. I say probably because it is possible that the tools would change the placement and routing on this path to guarantee that the arrival time would be lower than 5ns.
</div>


* Explore the **Report Noise** and the **Report Power** under the Open *Implemented Design*entry of the *Synthesis* tasks of the *Flow Navigator* pane. 

_____

## Step 3  - Simulate the Design ## 

### Step 3.1 ###

We will now simulate this design. However, to do that, we should reduce the sine wave frequency, so that the simulation does not take very long.

* First step will be to change constant values. Open the constant modules an reconfigure them to:
    * Constant (xlconstant_0) should have a Const Value = 4096
    * Constant (xlconstant_1) should have a Const Value = 8192
    * Constant (xlconstant_2) should have a Const Value = 1
    * Constant (xlconstant_3) should have a Const Value = 2


* In the *Flow Navigator*, click **Settings** again. Then select *IP* and click on the button **Clear Cache**. Click **OK**.


<div class="alert alert-block alert-warning">
<b>Attention:</b> OOC (Out-Of-Context) synthesis may exhibit bugs in some Vivado versions. Sometimes you change parameters in IPs of the Block Design and the IPs are not identified as out-of-date because changes are not detected and Vivado continues using the synthesis results in the cache. For that reason we may need to *Clear de Cache* to make sure the design is updated.
</div>


* Once all constants have been modified and the cache cleared, click **save** and then **Generate Block Design** to update your block design.


* **Run Synthesis** again and make sure it completes successfully.


* Click **Add Sources** and select *Add or create simulation sources*. Then add the testbench provided with design sources (`lab2_tb.vhd`). This testbench allows you to observe the generated signals both for sw=0 and sw=1. 


* In the *Flow Navigator*, click **Settings**. Then select *Simulation* and make sure that *Simulation top-module name* is set to `lab2_tb`. Also, in the Simulation tab, observe that the *Simulation Run Time* is 1000ns and change it to 2000ns. Click **OK**.


* With the simulation settings properly configured, you can launch Vivado simulator to perform a behavioral simulation of the design. In the *Flow Navigator*, click **Run Simulation > Run Behavioral Simulation**. In the Vivado IDE, the simulator GUI opens after successfully parsing and compiling the design.


<div class="alert alert-block alert-info">
<b>Info:</b> Functional and timing simulations are available post-synthesis and post-implementation. Those simulations take into consideration logic and routing delays but are outside the scope of this lab work.
</div>


<div class="alert alert-block alert-warning">
<b>Attention:</b> if you change the testbench or the design, you should *Reset Simulation* (right-click on **sim_1** file on the *Sources* window) to make sure all the changes are taken into account and the simulation results reflect those changes.
</div>



### Step 3.2 ###

By default, the Vivado simulator adds only the top-level HDL objects display in the Waveform window. In this step we will add and monitor design signals to observe the function of the circuit.

* In the *Scope* window,**expand** the `lab2_tb` hierarchy. Note that the device under test (**dut**) includes the lab2 instance (**lab2_i**) defined by the design unit `lab2_wrapper`. HDL scopes are modules, functions, tasks, processes, or blocks in the design.


* In the *Scope* window, click to **select** the module `sine_0`. The current scope changes and the *Objects* window updates with all the signals and constants of the selected scope.


* From the *Objects* window, select signal **sine_out** and add it into *Waveform* window using one of the following methods:
    * Drag and drop the selected signals into the *Waveform* window.
    * Right-click on the signal to open the popup menu, and select *Add to Wave Window*


<div class="alert alert-block alert-info">
<b>Info:</b> You can select multiple signals by holding down the CTRL key during selection.
</div>


* Repeat the last steps to add the output signals of the modules `ftrigger_0` and `counter_0`, namelly the signals **freq_trig** and **cnt_out**, to the *Waveform* window.


* The sine signal you are monitoring is an analog signal, which can be better seen in *Analog wave* mode. In the *Waveform* window, **select** the sine signal, **right-click** to open the popup menu, and select `Waveform Style > Analog`.


* **Right-click** the sine signal again to open the popup menu, and select `Radix > Unsigned Decimal` as this signal was defined that way in HDL. 


<div class="alert alert-block alert-info">
<b>Info:</b> If the sine wave samples were represented in two’s complement you would have to select `Radix > Signed Decimal` so that the binary information would be correctly interpreted by the analog wave viewer.
</div>


* **Reset** the simulation and **Run for a specified time** and set it to **30ms**, using the simulation menu buttons shown in the figure. Observe the sine signal output in the waveform. You can see a bit more than one period of the higher frequency sine wave (sw=1) and then a little of the lower frequency sin wave (sw=0).

![Figure](../lab2/lab2_figures/fig9.png)

<div class="alert alert-block alert-info">
<b>Info:</b> The Wave window can be undocked from Main window layout to view it as standalone. To do that click the Float button in the top right corner of the Waveform Configuration window.
</div>


* Display the whole time spectrum in the *Waveform* window by clicking the **Zoom Fit** button. You should be able to see a complete cycle for the low frequency sine wave.


![Figure](../lab2/lab2_figures/fig10.png)


* View the **cnt_out** and **freq_trig** waveforms in detail by zooming into the *Waveform* window. When you zoom into the waveform, you can use the horizontal and vertical scroll bars to pan down the full waveform.


* In the *Waveform* window, **right-click** to open the popup menu and select **New Divider**. Set the name to *Internal Signals*.


<div class="alert alert-block alert-info">
<b>Info:</b> Dividers let you create visual breaks between signals or groups of signals to more easily identify related objects. You can change divider names at any time by highlighting the divider name and selecting the Rename command from the popup menu, or change the color with Divider Color.
</div>


### Step 3.3 ###

We will now customize and save the Waveform configuration to reuse in future simulation runs. The Waveform configuration file defines the displayed signals, and the display characteristics of those signals.


* In the *Waveform* window, click the **Settings** button on the top right corner of the *waveform* window. A dialog box opens in a *General* tab.


* Ensure the *Default Radix* is set to *Hexadecimal*. This defines the default number format for all signals in the Waveform window. The radix can also be set for individual objects in the Waveform window to override the default.


* Select the *Draw Waveform* Shadow to enable or disable the shading under the signal waveform. By default, a waveform is shaded under the high transitions to make it easier to recognize the transitions and states in the Waveform window. You can also enable or disable signal indices, so that each signal or group of signals is identified with an index number in the Waveform window.


* Check or uncheck the Show signal indices check box to enable or disable the signal list numbering.


* Select the **Colors** tab and examine the Waveform Color Options. When you are satisfied, close de *Settings* pane.


<div class="alert alert-block alert-info">
<b>Info:</b> You can configure the coloring for elements of the Waveform window to customize the look and feel. You can specify custom colors to display waveforms of certain values, so you can quickly identify signals in an unknown state, or an uninitialized state. The Waveform window configures with your preferences. You can save the current waveform configuration so it is available for use in future Vivado simulation sessions.
</div>


* In the *Waveform* window sidebar menu, select the **Save Waveform Configuration** button. Save  it to the project folder with the default filename `lab2_tb_behav.wcfg`. Click **Save** and the **Yes**. The file is added to the project simulation fileset, sim_1, for archive purposes.

 
<div class="alert alert-block alert-info">
<b>Info:</b> You can also load a previously saved waveform configuration file using the File > Open Waveform Configuration command.
</div>



____________

## Homework Challenge ## 

### Challenge 1 ###

The DDS generator implemented in this lab work includes a counter and a sine LUT. The maximum counting value is constant. However, we can change the sine frequency using the counter enable signal (trigger output). The frequency of the sine wave will be inverselly proportional to the enable signal period.

The challenge here would be for you to implement a classic DDS arquitecture, with an accumulator replacing the counter. To change the clock frequency we have to change the accumulation step (B) - the bigger the B the higher would be sine wave frequency. As a tip I provide here a figure of the Block Design:

![Figure](../lab2/lab2_figures/fig11.png)

Also note that if you change the testbench or the design, you should reset the simulation (as shown in the previous figure) to make sure all the changes are taken into account and the simulation results reflect those changes.


* Start by creating a new project with name homew2, create a Block Design with an Accumulator from the Xilinx IP core library and add the following provided files:
    * sine_hw.v - source file for the sin LUT IP
    * sine_values_hw.dat - data file with sine values
    * homew2_tb.vhd - testbench for this challenge


* Simulate the design and verify its functionality.



### Challenge 2 ###

Add a *DDS Compiler* IP core from the Xilinx library and configure it to behave as a sine and cosine wave generator. Parameters and characteristcs should be similar to the synthesizer developed in Challenge 1. Update the block design with this new module, configure it and adapt the testbench so that you can simulate the design and verify its functionality.